# Ransac affine for point cloud alignment
In this tutorial we will learn to align several point clouds using two variants of the ransac affine and ICP algorithm.

Trying to implement other better non-rigid methods.

We begin with loading the required modules.

In [ ]:
# import pools
import os
import sys
import itertools
from math import pi, sin, cos, sqrt
import numpy as np
import matplotlib.pyplot as plt

import cv2
from matplotlib import cm
from scipy import ndimage
import scipy.io
from skimage import data
from skimage.io import imread, imsave
import pandas as pd
from scipy.spatial import cKDTree
from cv2 import estimateTranslation3D
import tifffile
import seaborn as sns

# import bigstream library
import zarr
from bigstream import features
from bigstream import features1
from bigstream import ransac
from bigstream import affine
from bigstream import affine1
from bigstream import transform
from fishspot.filter import white_tophat
from fishspot.detect import detect_spots_log

# napari
%gui qt5
import napari
viewer = napari.view_image(data.astronaut(), rgb=True)
napari.run()

We begin with loading the required modules for ransac.

In [ ]:
# ransac from bigstream
def blob_detection(
    image,
    min_blob_radius,
    max_blob_radius,
    **kwargs,
):
    """
    """

  #  wth = white_tophat(image, max_blob_radius)
    spots = detect_spots_log(
        image,
        min_blob_radius,
        max_blob_radius,
        **kwargs,
    ).astype(int)
    intensities = image[spots[:, 0], spots[:, 1], spots[:, 2]]
    return np.hstack((spots[:, :3], intensities[..., None]))

def get_spot_context(image, spots, vox, radius):
    """
    """

    # s[0] need to be pixels in images, spot is phsyical distance/um, and vox is um/pixel
    output = []
    for spot in spots:
        s = (spot/vox).astype(int)
        w = image[s[0]-radius:s[0]+radius+1,
                  s[1]-radius:s[1]+radius+1,
                  s[2]-radius:s[2]+radius+1]
        output.append( [spot, w] )
    return output    

def pairwise_correlation(A, B):
    """
    """

    # grab and flatten context
    a_con = np.array( [a[1].flatten() for a in A] )
    b_con = np.array( [b[1].flatten() for b in B] )

    # get means and std for all contexts, center contexts
    a_mean, a_std = _stats(a_con)
    b_mean, b_std = _stats(b_con)
    a_con = a_con - a_mean[..., None]
    b_con = b_con - b_mean[..., None]

    # compute pairwise correlations
    corr = np.matmul(a_con, b_con.T)
    corr = corr / a_std[..., None]
    corr = corr / b_std[None, ...]
    corr = corr / a_con.shape[1]

    # contexts with no variability are nan, set to 0
    corr[np.isnan(corr)] = 0
    return corr

def _stats(arr):
    """
    """

    # compute mean and standard deviation along columns
    arr = arr.astype(np.float64)
    means = np.mean(arr, axis=1)
    sqr_means = np.mean(np.square(arr), axis=1)
    stddevs = np.sqrt( sqr_means - np.square(means) )
    return means, stddevs

def match_points(A, B, scores, threshold):
    """
    """

    # split positions from context
    a_pos = np.array( [a[0] for a in A] )
    b_pos = np.array( [b[0] for b in B] )

    # get highest scores above threshold
    best_indcs = np.argmax(scores, axis=1)
    a_indcs = range(len(a_pos))
    keeps = scores[(a_indcs, best_indcs)] > threshold

    # return positions of corresponding points
    return a_pos[keeps, :3], b_pos[best_indcs[keeps], :3]

def ransac_align_points(
    pA, pB, threshold, diagonal_constraint=0.75, default=np.eye(4)[:3],
):
    """
    """

    # sensible requirement of 51 or more spots to compute ransac affine
    if len(pA) <= 4 or len(pB) <= 4:
        if default is not None:
            print("Insufficient spot matches for ransac, returning default identity")
            return default
        else:
            raise ValueError("Insufficient spot matches for ransac, need more than 4")

    # compute the affine
    r, Aff, inline = cv2.estimateAffine3D(pA, pB, ransacThreshold=threshold, confidence=0.999)

#     print(np.diag(Aff))
#     print(Aff)
#     print(diagonal_constraint)
    # rarely ransac just doesn't work (depends on data and parameters)
    # sensible choices for hard constraints on the affine matrix
    if np.any( np.diag(Aff) < diagonal_constraint ):
        if default is not None:
            print("Degenerate affine produced, returning default identity")
            return default
        else:
            raise ValueError("Degenerate affine produced, ransac failed")

    return Aff

In [ ]:
def eucldist(coords1, coords2):
    """ Calculates the euclidean distance between 2 lists of coordinates. """
    dist = np.zeros(len(coords1))
    i = 0
    for (x, y) in zip(coords1, coords2):
        p1 = x
        p2 = y
        squared_dist = (p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2
        dist[i] = np.sqrt(squared_dist)
        i = i+1
    return dist

In [ ]:
def get_spot_inside(all_spots,segmentation_mini,fix_spacing,ROI_fixed,cc):
    """remove spots outside of segmenation_mini
       change spot location back to segmentation mask in pixels
    """
    fix_spots_mini = all_spots[:,:3] / fix_spacing * [0.5,0.25,0.25] + cc
    spot_mini = np.zeros(len(fix_spots_mini))
    rounded_spot = fix_spots_mini.astype('int') 
    for i in range(0, len(fix_spots_mini)):          
        Coord = rounded_spot[i]
        if Coord[0]<0: Coord[0] = 0
        if Coord[1]<0: Coord[1] = 0
        if Coord[2]<0: Coord[2] = 0
        if Coord[0]>segmentation_mini.shape[0]: Coord[0] = segmentation_mini.shape[0]
        if Coord[1]>segmentation_mini.shape[1]: Coord[1] = segmentation_mini.shape[1]            
        if Coord[2]>segmentation_mini.shape[2]: Coord[2] = segmentation_mini.shape[2]
        idx = segmentation_mini[Coord[0]-1, Coord[1]-1, Coord[2]-1]   # roi id
        if idx == ROI_fixed:
            spot_mini[i] = idx  # add ROI number  
    spots_in = all_spots[np.where(spot_mini==ROI_fixed)]
    spots_out = all_spots[np.where(spot_mini==0)]

    ns = spots_in.shape[0]
    print(f'Image: found {ns} key points inside ROI')
    print(f'Image: found {spots_out.shape[0]} key points outside ROI')
    return spots_in

In [ ]:
# compute distance of paired spot cloud
def cloud_distance(spot_fix,spot_mov):
    """
    """
    c0=spot_fix[:,:3].copy()
    c1=spot_mov[:,:3].copy()
    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
    
#     ## find the smallest repeated value location, and delete the others
#     for i in range(Idx_unique.shape[0]):  
#         Loc_rep=np.where(idx2==Idx_unique[i])  # multiple loc, but should repeat for only once   
#     #     print(Loc_rep[0])
#         A=dist2[Loc_rep[0],Loc_rep[1]]
#         minposition = min(A)
#         Loc_min = np.where(A==minposition)[0]
#     #     print(Loc_min)
#     #     Loc_rep_min=Loc_rep[0][Loc_min[0]]    
#         Loc_rep_nouse=np.delete(range(len(Loc_rep[0])),Loc_min)
#         dist2[Loc_rep[0][Loc_rep_nouse],Loc_rep[1][Loc_rep_nouse]]=neighbor_radius*2 
#     ## find the results that are less than radius; used later column data 
#     # when only first row is not exist use latter column, or just dispose it. 
#     co_loc=np.where(dist2>neighbor_radius)

#     for j in range(dist2.shape[0]):
#          if dist2[j,0] < neighbor_radius:
#                 dist2[j,1] = neighbor_radius*2
#     for j in range(dist2.shape[0]):            
#          if dist2[j,0] <neighbor_radius or dist2[j,1] <neighbor_radius:
#                 dist2[j,2] = neighbor_radius*2       
#     row_c1 = np.where(dist2<neighbor_radius)
# #     print(len(row_c1[0]))

#     # lipo spot_c1 is row_c1
#     pBind = row_c1[0]
#     # print(idx2)
#     # lipo spot_c1 is idx2
#     pAind = [(idx2[row_c1[0][x], row_c1[1][x]]) for x in range(len(row_c1[0]))]
#     lipo_c0 = spot_c0[pAind]
#     lipo_c1 = spot_c1[pBind]

# #     print(np.unique(pAind).shape)
    
#     true_pos_c0 = np.delete(spot_c0, pAind, axis=0)
#     true_pos_c1 = np.delete(spot_c1, pBind, axis=0) #true
    
    
    return dist2[:,0]

### Colocalization filter 

In [ ]:
def colocalization(spot_c0,spot_c1,neighbor_radius):
    # Fix and Mov true spots
#     neighbor_radius = math.sqrt(3*3*3) #3.464
    #vox=[0.23,0.23,0.38]
    c0=spot_c0[:,:3].copy()
    c1=spot_c1[:,:3].copy()

    kdtree_c0 = cKDTree(c0)
    kdtree_c1 = cKDTree(c1)
    neighbors = kdtree_c0.query_ball_tree(kdtree_c1, neighbor_radius)
    # neighbors1 = kdtree_c0.query_ball_point(c1,neighbor_radius) #1 is all unique idx, 2 is the location
    # neighbors2 = kdtree_c0.sparse_distance_matrix(kdtree_c1, neighbor_radius)
    dist2,idx2 = kdtree_c0.query(c1, k=3)
    [Idx_unique, I] = np.unique(idx2,return_index=True) 
#     print(len(dist2[:,0])*3)
    
    ## find the smallest repeated value location, and delete the others
    for i in range(Idx_unique.shape[0]):  # should repeat for only once
    #     print(Idx_unique[i])
        Loc_rep=np.where(idx2==Idx_unique[i])    
    #     print(Loc_rep[0])
        A=dist2[Loc_rep[0],Loc_rep[1]]
        minposition = min(A)
        Loc_min = np.where(A==minposition)[0]
    #     print(Loc_min)
    #     Loc_rep_min=Loc_rep[0][Loc_min[0]]    
        Loc_rep_nouse=np.delete(range(len(Loc_rep[0])),Loc_min)
        dist2[Loc_rep[0][Loc_rep_nouse],Loc_rep[1][Loc_rep_nouse]]=neighbor_radius*2 
    ## find the results that are less than radius; used later column data 
    # when only first row is not exist use latter column, or just dispose it. 
    co_loc=np.where(dist2>neighbor_radius)

    for j in range(dist2.shape[0]):
         if dist2[j,0] < neighbor_radius:
                dist2[j,1] = neighbor_radius*2
    for j in range(dist2.shape[0]):            
         if dist2[j,0] <neighbor_radius or dist2[j,1] <neighbor_radius:
                dist2[j,2] = neighbor_radius*2       
    row_c1 = np.where(dist2<neighbor_radius)
#     print(len(row_c1[0]))

    # lipo spot_c1 is row_c1
    pBind = row_c1[0]
    # print(idx2)
    # lipo spot_c1 is idx2
    pAind = [(idx2[row_c1[0][x], row_c1[1][x]]) for x in range(len(row_c1[0]))]
    lipo_c0 = spot_c0[pAind]
    lipo_c1 = spot_c1[pBind]

#     print(np.unique(pAind).shape)
    
    true_pos_c0 = np.delete(spot_c0, pAind, axis=0)
    true_pos_c1 = np.delete(spot_c1, pBind, axis=0) #true

    if spot_c0.shape[0]>0:
        P1 = (lipo_c0.shape[0] / spot_c0.shape[0])*100   # % mov spots from  previous images  /spot_c0.shape
    else:
        P1 = 0
        
    if spot_c1.shape[0]>0:
        P2 = (lipo_c1.shape[0] / spot_c1.shape[0])*100  # % fixed spots can be found in later mov images  /spot_c0.shape
    else:
        P2 = 0

    print('% P1: ',str(P1) + ';  % P2: ',str(P2)) 
    Dist = eucldist(lipo_c0,lipo_c1)

    return lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,Dist

In [ ]:
## ICP
def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nx3 array of points
        dst: Nx3 array of points
    Output:
        distances: Euclidean distances (errors) of the nearest neighbor
        indecies: dst indecies of the nearest neighbor
    '''

    indecies = np.zeros(src.shape[0], dtype=np.int)
    distances = np.zeros(src.shape[0])
    for i, s in enumerate(src):
        min_dist = np.inf
        for j, d in enumerate(dst):
            dist = np.linalg.norm(s-d)
            # 找出与src[i]距离最近的dst[j]
            if dist < min_dist:
                min_dist = dist
                indecies[i] = j
                distances[i] = dist
    return distances, indecies  

def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform between corresponding 3D points A->B
    Input:
      A: Nx3 numpy array of corresponding 3D points
      B: Nx3 numpy array of corresponding 3D points
    Returns:
      T: 4x4 homogeneous transformation matrix
      R: 3x3 rotation matrix
      t: 3x1 column vector
    '''
    assert len(A) == len(B)

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0) 
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    W = np.dot(BB.T, AA)
    U, s, VT = np.linalg.svd(W, full_matrices=True, compute_uv=True)
    R = np.dot(U, VT)

    # special reflection case
    if np.linalg.det(R) < 0:
        VT[2,:] *= -1
        R = np.dot(U, VT)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t

    return T, R, t
       
def icp(A, B, init_pose = None, max_iterations=200, tolerance=0.0001):
    '''
    The Iterative Closest Point method
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    #  select points 
    A,B,_,_,dist = colocalization(A,B,3)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0]))  #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T)  # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T)
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0
    distances_iter = np.zeros((max_iterations,1))
    for i in range(max_iterations):
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)

#         AAA = np.where(distances<3)
#         BBB = indices[AAA]
#         src = src[0:3,AAA]  # A.T shape (3,20)
#         dst = dst[0:3,BBB]
#         A = A[AAA,0:3]
        
        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  #将dst[]按照indices排序
#         T,_,_ = best_fit_transform(src[0:3,AAA].T, dst[0:3,BBB].T)  #将dst[]按照indices排序

        # update the current source
    # refer to "Introduction to Robotics" Chapter2 P28. Spatial description and transformations
        src = np.dot(T, src)

        # check error
        mean_error = np.sum(distances) / distances.size
#         print(f'mean_error: {mean_error}')
        if abs(prev_error-mean_error) < tolerance:
            break
        prev_error = mean_error
        distances_iter[i] = mean_error
    
#     fig=plt.figure(dpi=120,figsize=(2,3))
#     plt.plot(distances_iter[:20])
#     sns.despine() 
#     plt.xlabel('Spots:'+ str(distances_iter[:20].shape[0]))
#     plt.ylabel('Distance/pixel')
#     ave=np.average(distances_iter[:20])
#     plt.title(str(float('%.2f' % ave)))
#     plt.show()
    
#     print(f'mean_error: {mean_error}')
    # calculcate final tranformation
    T,_,_ = best_fit_transform(A, src[0:3,:].T)

    return T, distances     

# use import spots or not. If yes, the spot should align with images checked with Napari

In [ ]:
hAir = 0
# Adapt ROI from images or from hAirlocalize
if hAir < 1:
    print(f'hAirlocalize points')
    # read spot data into memory as numpy arrays  Airlocolize spots data in xyz order: physical distance
    spotdir = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/spots/R2_c3_ROI.txt'
    spot_fix=np.loadtxt(spotdir, delimiter=',')
    fixed_spots1 = spot_fix[spot_fix[:,4] == i][:,:3]
    spotdir = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/spots/R1_c3_ROI.txt'
    spot_mov=np.loadtxt(spotdir, delimiter=',')
    moving_spots1 = spot_mov[spot_mov[:,4] == i][:,:3]
    # change to zyx order
    fixed_spots11 = np.transpose(np.array([fixed_spots1[:,2],fixed_spots1[:,1],fixed_spots1[:,0]]))
    moving_spots11 = np.transpose(np.array([moving_spots1[:,2],moving_spots1[:,1],moving_spots1[:,0]]))
    # convert to physical units
    ccc = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
    fix_spots = (fixed_spots11 - ccc * fix_spacing)
    fix_spots_new = fix_spots
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')

    dd = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
#     mov_spots_h = (moving_spots11/fix_spacing - dd).astype(int)
    mov_spots = (moving_spots11 - dd * mov_spacing)
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')

In [ ]:
def ROI_affine(segmentation1,segmentation2,ROI_fixed,ROI_moving,
               fixed_image,moving_image,
               threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold):
    """
    """
#     threshold_fixed=0.0001 #0.001 for c3 fix mov 
#     threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
#     cc_r=12 
#     match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
#     align_threshold=3.0
    
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    # fix_ds_spacing = fix_spacing/zoom
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
    seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
    neighbor_radius1 = 3
    zoom=[2,4,4]

    AA=np.where(segmentation1==ROI_fixed)
    fixed_ROI = fixed_image[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
              min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
              min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
    print(fixed_ROI.shape)

    # enlarge the moving image ROI view
    BB=np.where(segmentation2==ROI_moving)
    En_pixels = 15 # Enlarged pixels 
    b1 = min(BB[0])*zoom[0] - En_pixels
    b2 = max(BB[0])*zoom[0] + En_pixels
    b3 = min(BB[1])*zoom[1] - En_pixels
    b4 = max(BB[1])*zoom[1] + En_pixels
    b5 = min(BB[2])*zoom[2] - En_pixels
    b6 = max(BB[2])*zoom[2] + En_pixels
    #Edges evaluation
    def ROI_edge(b1,b2,b3,b4,b5,b6):        
        if b1 <0:
            b1 = 0
        if b3 <0:
            b3 = 0  
        if b5 <0:
            b5 = 0 
        if b2 >= moving_image.shape[0]:
            b2 = moving_image.shape[0] -1
        if b4 >= moving_image.shape[1]:
            b4 = moving_image.shape[1] -1 
        if b6 >= moving_image.shape[2]:
            b6 = moving_image.shape[2] -1     
        return b1,b2,b3,b4,b5,b6

    b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
    moving_ROI = moving_image[b1:b2,b3:b4,b5:b6]
    
    print(moving_ROI.shape)
    fix_ds = fixed_ROI
    mov_ds = moving_ROI
    
    # get spots in pixels
#     print('Getting key points')
    fix_spots = blob_detection(
        fix_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_fixed, exclude_border=cc_radius,
    )
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')
    
    mov_spots = blob_detection(
        mov_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_moving, exclude_border=cc_radius,
    )
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')
    # sort
    sort_idx = np.argsort(fix_spots[:, 3])[::-1]
    fix_spots = fix_spots[sort_idx, :3][:nspots]
    sort_idx = np.argsort(mov_spots[:, 3])[::-1]
    mov_spots = mov_spots[sort_idx, :3][:nspots]
    # convert to physical units
    fix_spots = fix_spots * fix_spacing
    mov_spots = mov_spots * mov_spacing
    
    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    segmentation_mini=imread(seg_dir1+'mask_all_R2.tif')
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    fix_spots_new = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     mov_spots_new = get_spot_inside(mov_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    mov_spots_new = mov_spots

    # get contexts
    fix_spots0 = features.get_spot_context(
        fix_ds, fix_spots_new, fix_spacing, cc_r,
    )
    mov_spots0 = features.get_spot_context(
        mov_ds, mov_spots_new, mov_spacing, cc_r,
    )
    
    # get point correspondences may change to mutual information
    correlations = features.pairwise_correlation(
        fix_spots0, mov_spots0,
    )
    
    fix_spots1, mov_spots1 = features.match_points(
        fix_spots0, mov_spots0,
        correlations, match_threshold,
    )
    
    print(f'Found {fix_spots1.shape[0]} matched fixed points')

    global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)
#     print(global_affine)    
    inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)
    
#     print("Affine invtransformation matrix:\n", inv_affine)
    points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)

    warp_spots = points.dot(inv_affine.T)
#     print(f'Found {warp_spots.shape[0]} warp points')
    
    mov_affine = transform.apply_global_affine(
    fix_ds, mov_ds,
    fix_spacing, fix_spacing,
    global_affine,)
    
    print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(fix_spots_new,mov_spots_new))}')
    
    print('colocalization of all inside spots after ransac affine') 
    spot_fix = fix_spots_new
#     spot_mov = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    spot_mov = warp_spots
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(spot_fix,spot_mov))}')
    
    print('registration of all inside spots with ICP')
    A = spot_fix[:,:3]
    B = spot_mov[:,:3]
    print(f'{A.shape[0]} icp fix points')
    print(f'{B.shape[0]} icp mov points')
    
    Transform, distances1 = icp(A, B)
    inv_Transform, distances2 = icp(B, A)
    print(f'Distance_AB: {np.mean(distances1)}')
    print(f'Distance_BA: {np.mean(distances2)}')
    np.set_printoptions(precision=3,suppress=True)
#     print (Transform)
    # functions for applying transforms are in bigstream.transform. apply the ICP affine to the moved image
    ICP_affine = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        Transform,)
    ICP_affine_inv = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        inv_Transform,)

    #     print (inv_Transform)
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)
    C = p.dot(inv_Transform.T)
    # Test ICP distance change. Minor increased performance
    spot_fix = fix_spots
    warp_spots_new = get_spot_inside(C,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     warp_spots_new = C
    spot_mov = warp_spots_new
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
    distances3 = cloud_distance(spot_fix,spot_mov)
    print(f'Distance: {np.mean(distances3)}')
    
    return np.mean(eucldist(lipo_c0,lipo_c1)),global_affine, mov_affine, fixed_ROI, moving_ROI, inv_affine,inv_Transform,fix_spots_new,mov_spots_new,spot_fix,warp_spots_new,ICP_affine,ICP_affine_inv,cc

In [ ]:
def ROI_affine_aligntrack(segmentation1,segmentation2,ROI_fixed,ROI_moving,
               fixed_image,moving_image,
               threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
               track_affine,global_affine_0,inv_Transform_0):
    """
    """
#     threshold_fixed=0.0001 #0.001 for c3 fix mov 
#     threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
#     cc_r=12 
#     match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
#     align_threshold=3.0
    
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    # fix_ds_spacing = fix_spacing/zoom
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
    seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
    neighbor_radius1 = 3
    zoom=[2,4,4]
    
    AA=np.where(segmentation1==ROI_fixed)
    fixed_ROI = fixed_image[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
              min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
              min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
    print(fixed_ROI.shape)

   # enlarge the moving image ROI view
    BB=np.where(segmentation2==ROI_moving)
    En_pixels = 15 # Enlarged pixels 
    b1 = min(BB[0])*zoom[0] - En_pixels
    b2 = max(BB[0])*zoom[0] + En_pixels
    b3 = min(BB[1])*zoom[1] - En_pixels
    b4 = max(BB[1])*zoom[1] + En_pixels
    b5 = min(BB[2])*zoom[2] - En_pixels
    b6 = max(BB[2])*zoom[2] + En_pixels
    #Edges evaluation
    def ROI_edge(b1,b2,b3,b4,b5,b6):        
        if b1 <0:
            b1 = 0
        if b3 <0:
            b3 = 0  
        if b5 <0:
            b5 = 0 
        if b2 >= moving_image.shape[0]:
            b2 = moving_image.shape[0] -1
        if b4 >= moving_image.shape[1]:
            b4 = moving_image.shape[1] -1 
        if b6 >= moving_image.shape[2]:
            b6 = moving_image.shape[2] -1     
        return b1,b2,b3,b4,b5,b6

    b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
    moving_ROI = moving_image[b1:b2,b3:b4,b5:b6]
    
    print(moving_ROI.shape)
    fix_ds = fixed_ROI
        
    if track_affine > 0:             ###################################  
#         mov_ds = moving_ROI        
        mov_affine_0 = transform.apply_global_affine(
        fix_ds, moving_ROI,
        fix_spacing, fix_spacing,
        global_affine_0,)
        
        mov_ds = transform.apply_global_affine(
        fix_ds, mov_affine_0,
        fix_spacing, fix_spacing,
        inv_Transform_0,)       
    else:
        mov_ds = moving_ROI
              
    # get spots in pixels
#     print('Getting key points')
    fix_spots = blob_detection(
        fix_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_fixed, exclude_border=cc_radius,
    )
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')

    mov_spots = blob_detection(
        mov_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_moving, exclude_border=cc_radius,
    )
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')
    # sort
    sort_idx = np.argsort(fix_spots[:, 3])[::-1]
    fix_spots = fix_spots[sort_idx, :3][:nspots]
    sort_idx = np.argsort(mov_spots[:, 3])[::-1]
    mov_spots = mov_spots[sort_idx, :3][:nspots]
    
    # convert to physical units
    fix_spots = fix_spots * fix_spacing
    mov_spots = mov_spots * mov_spacing

    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    segmentation_mini=imread(seg_dir1+'mask_all_R2.tif')
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    
    fix_spots_new = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     mov_spots_new = get_spot_inside(mov_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    mov_spots_new = mov_spots
    
    print('registration of all inside spots with ICP')
    A = fix_spots_new[:,:3]
    B = mov_spots_new[:,:3]
    print(f'{A.shape[0]} icp fix points')
    print(f'{B.shape[0]} icp mov points')
    Transform, distances1 = icp(A, B)
    inv_Transform, distances2 = icp(B, A)
    print(f'Distance_AB: {np.mean(distances1)}')
    print(f'Distance_BA: {np.mean(distances2)}')
    np.set_printoptions(precision=3,suppress=True)
    # functions for applying transforms are in bigstream.transform. apply the ICP affine to the moved image
    ICP_affine = transform.apply_global_affine(
        fix_ds, mov_ds,
        fix_spacing, fix_spacing,
        Transform,)
    ICP_affine_inv = transform.apply_global_affine(
        fix_ds, mov_ds,
        fix_spacing, fix_spacing,
        inv_Transform,)
    #     print (inv_Transform)
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)
    C = p.dot(inv_Transform.T)[:,:3]
    print(fix_spots_new.shape)
    print(C.shape)
    
#     # Test ICP distance change. Minor increased performance
    
#         # get contexts
#     fix_spots0 = features.get_spot_context(
#         fix_ds, fix_spots_new, fix_spacing, cc_r,
#     )
#     mov_spots0 = features.get_spot_context(
#         mov_ds, C, mov_spacing, cc_r,
#     )

#     # get point correspondences may change to mutual information
#     correlations = features.pairwise_correlation(
#         fix_spots0, mov_spots0,
#     )

#     fix_spots1, mov_spots1 = features.match_points(
#         fix_spots0, mov_spots0,
#         correlations, match_threshold,
#     )
    fix_spots1, mov_spots1,true_pos_c0,true_pos_c1,dist = colocalization(A,C,neighbor_radius1)

    print(f'Found {fix_spots1.shape[0]} matched fixed points')

    global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)
#     print(global_affine)    
    inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)

    print("Affine invtransformation matrix:\n", inv_affine)
    points = np.append(C, np.ones((C.shape[0],1)), axis=1)
    warp_spots = points.dot(inv_affine.T)

    mov_affine = transform.apply_global_affine(
    fix_ds, ICP_affine,
    fix_spacing, fix_spacing,
    global_affine,)

    print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(fix_spots_new,mov_spots_new))}')

    print('colocalization of inside spots after ransac affine') 
    spot_fix = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     spot_mov = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    warp_spots_new = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    spot_mov = warp_spots_new 
    
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(lipo_c0,lipo_c1))}')
    
#     warp_spots_new = get_spot_inside(C,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     spot_mov = warp_spots_new 
#     lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
#     distances3 = cloud_distance(spot_fix,spot_mov)
#     print(f'Distance: {np.mean(distances3)}')
                             
    return np.mean(eucldist(lipo_c0,lipo_c1)),global_affine, mov_affine, fixed_ROI, mov_ds, moving_ROI, inv_affine,Transform,inv_Transform,fix_spots_new,mov_spots_new,spot_fix,warp_spots,warp_spots_new,ICP_affine,ICP_affine_inv,cc

In [ ]:
def ROI_affine_aligntrack1(segmentation1,segmentation2,ROI_fixed,ROI_moving,
               fixed_image,moving_image,
               threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
               track_affine,global_affine_0,inv_Transform_0):
    """
    """
#     threshold_fixed=0.0001 #0.001 for c3 fix mov 
#     threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
#     cc_r=12 
#     match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
#     align_threshold=3.0
    
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    # fix_ds_spacing = fix_spacing/zoom
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
    seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
    neighbor_radius1 = 3
    zoom=[2,4,4]
    
    AA=np.where(segmentation1==ROI_fixed)
    fixed_ROI = fixed_image[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
              min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
              min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
    print(fixed_ROI.shape)

   # enlarge the moving image ROI view
    BB=np.where(segmentation2==ROI_moving)
    En_pixels = 15 # Enlarged pixels 
    b1 = min(BB[0])*zoom[0] - En_pixels
    b2 = max(BB[0])*zoom[0] + En_pixels
    b3 = min(BB[1])*zoom[1] - En_pixels
    b4 = max(BB[1])*zoom[1] + En_pixels
    b5 = min(BB[2])*zoom[2] - En_pixels
    b6 = max(BB[2])*zoom[2] + En_pixels
    #Edges evaluation
    def ROI_edge(b1,b2,b3,b4,b5,b6):        
        if b1 <0:
            b1 = 0
        if b3 <0:
            b3 = 0  
        if b5 <0:
            b5 = 0 
        if b2 >= moving_image.shape[0]:
            b2 = moving_image.shape[0] -1
        if b4 >= moving_image.shape[1]:
            b4 = moving_image.shape[1] -1 
        if b6 >= moving_image.shape[2]:
            b6 = moving_image.shape[2] -1     
        return b1,b2,b3,b4,b5,b6

    b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
    moving_ROI = moving_image[b1:b2,b3:b4,b5:b6]
    
    print(moving_ROI.shape)
    fix_ds = fixed_ROI
        
    if track_affine > 0:             ###################################  
#         mov_ds = moving_ROI        
        mov_affine_0 = transform.apply_global_affine(
        fix_ds, moving_ROI,
        fix_spacing, fix_spacing,
        global_affine_0,)
        
        mov_ds = transform.apply_global_affine(
        fix_ds, mov_affine_0,
        fix_spacing, fix_spacing,
        inv_Transform_0,)       
    else:
        mov_ds = moving_ROI
              
    # get spots in pixels
#     print('Getting key points')
    fix_spots = blob_detection(
        fix_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_fixed, exclude_border=cc_radius,
    )
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')

    mov_spots = features1.blob_detection(
        mov_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_moving, exclude_border=cc_radius,
    )
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')
    # sort
    sort_idx = np.argsort(fix_spots[:, 3])[::-1]
    fix_spots = fix_spots[sort_idx, :3][:nspots]
    sort_idx = np.argsort(mov_spots[:, 3])[::-1]
    mov_spots = mov_spots[sort_idx, :3][:nspots]
    
    # convert to physical units
    fix_spots = fix_spots * fix_spacing
    mov_spots = mov_spots * mov_spacing

    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    segmentation_mini=imread(seg_dir1+'mask_all_R2.tif')
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    fix_spots_new = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     mov_spots_new = get_spot_inside(mov_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    mov_spots_new = mov_spots
    
    # get contexts
    fix_spots0 = features.get_spot_context(
        fix_ds, fix_spots_new, fix_spacing, cc_r,
    )
    mov_spots0 = features.get_spot_context(
        mov_ds, mov_spots_new, mov_spacing, cc_r,
    )

    # get point correspondences may change to mutual information
    correlations = features.pairwise_correlation(
        fix_spots0, mov_spots0,
    )

    fix_spots1, mov_spots1 = features.match_points(
        fix_spots0, mov_spots0,
        correlations, match_threshold,
    )
    print(f'Found {fix_spots1.shape[0]} matched fixed points')

    global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)
#     print(global_affine)    
    inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)

    print("Affine invtransformation matrix:\n", inv_affine)
    points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
    warp_spots = points.dot(inv_affine.T)

    mov_affine = transform.apply_global_affine(
    fix_ds, mov_ds,
    fix_spacing, fix_spacing,
    global_affine,)

    print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(fix_spots_new,mov_spots_new))}')

    print('colocalization of inside spots after ransac affine') 
    spot_fix = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     spot_mov = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     spot_mov = warp_spots
    ###################################
    spot_mov = mov_spots_new
    
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov, neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(spot_fix,spot_mov))}')

    print('registration of all inside spots with ICP')
    A = spot_fix[:,:3]
    B = spot_mov[:,:3]
    print(f'{A.shape[0]} icp fix points')
    print(f'{B.shape[0]} icp mov points')
    Transform, distances1 = icp(A, B)
    inv_Transform, distances2 = icp(B, A)
    print(f'Distance_AB: {np.mean(distances1)}')
    print(f'Distance_BA: {np.mean(distances2)}')
    np.set_printoptions(precision=3,suppress=True)
#     print (Transform)
    # functions for applying transforms are in bigstream.transform. apply the ICP affine to the moved image
    ICP_affine = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        Transform,)
    ICP_affine_inv = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        inv_Transform,)
    #     print (inv_Transform)
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)
    C = p.dot(inv_Transform.T)
    # Test ICP distance change. Minor increased performance
    warp_spots_new = get_spot_inside(C,segmentation_mini,fix_spacing,ROI_fixed,cc)
    spot_mov = warp_spots_new 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
    distances3 = cloud_distance(spot_fix,spot_mov)
    print(f'Distance: {np.mean(distances3)}')
                             
    return np.mean(distances1),global_affine, mov_affine, fixed_ROI, mov_ds, moving_ROI, inv_affine,Transform,inv_Transform,fix_spots_new,mov_spots_new,spot_fix,warp_spots,warp_spots_new,ICP_affine,ICP_affine_inv,cc

In [ ]:
def ROI_affine_aligntrack0(segmentation1,segmentation2,ROI_fixed,ROI_moving,
               fixed_image,moving_image,
               threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
               track_affine,global_affine_0,inv_Transform_0):
    """
    """
#     threshold_fixed=0.0001 #0.001 for c3 fix mov 
#     threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
#     cc_r=12 
#     match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
#     align_threshold=3.0
    
    fix_spacing=np.array([0.42,0.23,0.23])
    mov_spacing=fix_spacing
    # fix_ds_spacing = fix_spacing/zoom
    min_radius=6
    max_radius=12
    cc_radius=cc_r # used for radius pixel of context information.
    nspots=30000
    num_sigma_max=6
    seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
    neighbor_radius1 = 3
    zoom=[2,4,4]
    
    AA=np.where(segmentation1==ROI_fixed)
    fixed_ROI = fixed_image[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
              min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
              min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
    print(fixed_ROI.shape)

   # enlarge the moving image ROI view
    BB=np.where(segmentation2==ROI_moving)
    En_pixels = 15 # Enlarged pixels 
    b1 = min(BB[0])*zoom[0] - En_pixels
    b2 = max(BB[0])*zoom[0] + En_pixels
    b3 = min(BB[1])*zoom[1] - En_pixels
    b4 = max(BB[1])*zoom[1] + En_pixels
    b5 = min(BB[2])*zoom[2] - En_pixels
    b6 = max(BB[2])*zoom[2] + En_pixels
    #Edges evaluation
    def ROI_edge(b1,b2,b3,b4,b5,b6):        
        if b1 <0:
            b1 = 0
        if b3 <0:
            b3 = 0  
        if b5 <0:
            b5 = 0 
        if b2 >= moving_image.shape[0]:
            b2 = moving_image.shape[0] -1
        if b4 >= moving_image.shape[1]:
            b4 = moving_image.shape[1] -1 
        if b6 >= moving_image.shape[2]:
            b6 = moving_image.shape[2] -1     
        return b1,b2,b3,b4,b5,b6

    b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
    moving_ROI = moving_image[b1:b2,b3:b4,b5:b6]
    
    print(moving_ROI.shape)
    fix_ds = fixed_ROI
        
    if track_affine > 0:             ###################################  
#         mov_ds = moving_ROI        
        mov_affine_0 = transform.apply_global_affine(
        fix_ds, moving_ROI,
        fix_spacing, fix_spacing,
        global_affine_0,)
        
        mov_ds = transform.apply_global_affine(
        fix_ds, mov_affine_0,
        fix_spacing, fix_spacing,
        inv_Transform_0,)       
    else:
        mov_ds = moving_ROI
              
    # get spots in pixels
#     print('Getting key points')
    fix_spots = blob_detection(
        fix_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_fixed, exclude_border=cc_radius,
    )
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')

    mov_spots = features1.blob_detection(
        mov_ds, min_radius, max_radius,
        num_sigma=min(max_radius-min_radius, num_sigma_max),
        threshold=threshold_moving, exclude_border=cc_radius,
    )
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')
    # sort
    sort_idx = np.argsort(fix_spots[:, 3])[::-1]
    fix_spots = fix_spots[sort_idx, :3][:nspots]
    sort_idx = np.argsort(mov_spots[:, 3])[::-1]
    mov_spots = mov_spots[sort_idx, :3][:nspots]
    
    # convert to physical units
    fix_spots = fix_spots * fix_spacing
    mov_spots = mov_spots * mov_spacing

    # remove spots outside of segmenation_mini
    # change spot location back to segmentation mask in pixels
    segmentation_mini=imread(seg_dir1+'mask_all_R2.tif')
    AA=np.where(segmentation1==ROI_fixed)
    cc = [min(AA[0]),min(AA[1]),min(AA[2])]
    fix_spots_new = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     mov_spots_new = get_spot_inside(mov_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    mov_spots_new = mov_spots
    
    # get contexts
    fix_spots0 = features.get_spot_context(
        fix_ds, fix_spots_new, fix_spacing, cc_r,
    )
    mov_spots0 = features.get_spot_context(
        mov_ds, mov_spots_new, mov_spacing, cc_r,
    )

    # get point correspondences may change to mutual information
    correlations = features.pairwise_correlation(
        fix_spots0, mov_spots0,
    )

    fix_spots1, mov_spots1 = features.match_points(
        fix_spots0, mov_spots0,
        correlations, match_threshold,
    )
    print(f'Found {fix_spots1.shape[0]} matched fixed points')

    global_affine = ransac_align_points(fix_spots1, mov_spots1, align_threshold,)
#     print(global_affine)    
    inv_affine = ransac_align_points(mov_spots1, fix_spots1, align_threshold,)

    print("Affine invtransformation matrix:\n", inv_affine)
    points = np.append(mov_spots, np.ones((mov_spots.shape[0],1)), axis=1)
    warp_spots = points.dot(inv_affine.T)

    mov_affine = transform.apply_global_affine(
    fix_ds, mov_ds,
    fix_spacing, fix_spacing,
    global_affine,)

    print('colocalization of all spots before ransac affine') 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(fix_spots_new,mov_spots_new,neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(fix_spots_new,mov_spots_new))}')

    print('colocalization of inside spots after ransac affine') 
    spot_fix = get_spot_inside(fix_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
#     spot_mov = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
    spot_mov = warp_spots
    
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov, neighbor_radius1)  # return in pixel
    print(f'Distance: {np.mean(cloud_distance(spot_fix,spot_mov))}')

    print('registration of all inside spots with ICP')
    A = spot_fix[:,:3]
    B = spot_mov[:,:3]
    print(f'{A.shape[0]} icp fix points')
    print(f'{B.shape[0]} icp mov points')
    Transform, distances1 = icp(A, B)
    inv_Transform, distances2 = icp(B, A)
    print(f'Distance_AB: {np.mean(distances1)}')
    print(f'Distance_BA: {np.mean(distances2)}')
    np.set_printoptions(precision=3,suppress=True)
#     print (Transform)
    # functions for applying transforms are in bigstream.transform. apply the ICP affine to the moved image
    ICP_affine = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        Transform,)
    ICP_affine_inv = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        inv_Transform,)
    #     print (inv_Transform)
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)
    C = p.dot(inv_Transform.T)
    # Test ICP distance change. Minor increased performance
    warp_spots_new = get_spot_inside(C,segmentation_mini,fix_spacing,ROI_fixed,cc)
    spot_mov = warp_spots_new 
    lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,warp_spots_new,neighbor_radius1)  # return in pixel
    distances3 = cloud_distance(spot_fix,spot_mov)
    print(f'Distance: {np.mean(distances3)}')
                             
    return np.mean(distances1),global_affine, mov_affine, fixed_ROI, mov_ds, moving_ROI, inv_affine,Transform,inv_Transform,fix_spots_new,mov_spots_new,spot_fix,warp_spots,warp_spots_new,ICP_affine,ICP_affine_inv,cc

## Preparing the data
We load the images of 2 channels of two rounds

In [ ]:
%%time
# # file paths to bigstrem REGISTERED data N5 files(include deform)
# create Zarr file object using N5Stores
# Analyze the below images ## image is in zyx order
fix_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/stitching/export.n5/c0/s0'  # R2_CO
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
fixed = fix_zarr[...]
fix3_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/stitching/export.n5/c3/s0' # R2_C3
fix3_zarr = zarr.open(store=zarr.N5Store(fix3_path), mode='r')
fixed3 = fix3_zarr[...]
print(fixed3.shape)

# mov_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/registration/python/warped_pre10162021/warped/c0/s0'  # R1_CO
# mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
# moving = mov_zarr[...]
# mov3_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/registration/python/warped_pre10162021/warped/c3/s0' # R1_C3
# mov3_zarr = zarr.open(store=zarr.N5Store(mov3_path), mode='r')
# moving3 = mov3_zarr[...]

## file paths to RAW data N5 files
mov_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/stitching/export.n5/c0/s0' # R1_C3
mov3_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/stitching/export.n5/c3/s0' # R2_C3
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
mov3_zarr = zarr.open(store=zarr.N5Store(mov3_path), mode='r')
moving = mov_zarr[...]
moving3 = mov3_zarr[...]
print(moving3.shape)

### input:
##### enlarged fixed ROIs, fixed image, moving image;
##### fixed spot extraction parameters;
##### moved spot extraction parameters;

### output:
#### visualization for example ROIs
##### enlarged fixed ROIs_fixed image, moving image, affined image;
##### fixed spots, moved spots, affined spots by fishspots

#### analyze spot location, remove spots outside of ROI, relocate the xyz coordinate to the orginal images

### refine ransac_affine or use other methods, and validate

## Piece-wise affine 
### Select segmented ROI

In [ ]:
# # UNREGISTERED
zoom=[2,4,4]
seg_dir='E:/Maxprobe_analysis/R2_R1_3tm50/copytostephan/'
# segmentation1=imread(seg_dir+'mask_GAD1_R2.tif') # 
# segmentation2=imread(seg_dir+'mask_GAD1_R1.tif') # Before bigstream
segmentation2=imread(seg_dir+'mask_all_R1.tif') # Before bigstream

# # Registered and enlarged ROI
seg_dir1='E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/segmentation/'
# segmentation1=imread(seg_dir1+'mask_GAD111.tif')
segmentation1=imread(seg_dir+'mask_all_R2.tif')
# segmentation_mini=imread(seg_dir1+'mask_GAD11.tif')

# apply the global affine to the moving segmented ROI
# segmentation2_affine = transform.apply_global_affine()

# viewer.add_image(segmentation1,colormap='green',blending='additive') #load image data into napari
# viewer.add_image(segmentation2,colormap='red',blending='additive') #load image data into napari
# viewer.add_image(segmentation2_affine,colormap='magenta',blending='additive') #load image data into napari
# viewer.add_image(segmentation_mini,colormap='magenta',blending='additive') #load image data into napari

In [ ]:
# # all spots show: key spots           
# # affine alignment functions are in bigstream.affine

# fix_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/stitching/export.n5/c2/s2'  # R2_CO
# fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
# fix_ds = fix_zarr[...]

# mov_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/stitching/export.n5/c2/s2' # R1_C3
# mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
# mov_ds = mov_zarr[...]

# zoom=np.array([0.5,0.25,0.25])
# fix_spacing=np.array([0.42,0.23,0.23])
# fix_ds_spacing = fix_spacing/zoom

# from bigstream import affine
# # see below for explanation of parameters
# global_affine = affine.ransac_affine(
#     fix_ds, mov_ds,
#     fix_ds_spacing, fix_ds_spacing,
#     min_radius=12, max_radius=16, match_threshold=0.50,
#     cc_radius=12, 
# )

In [ ]:
# fix_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/stitching/export.n5/c2/s0'  # R2_CO
# fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
# fix_ds = fix_zarr[...]

# mov_path = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/stitching/export.n5/c2/s0' # R1_C3
# mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
# mov_ds = mov_zarr[...]

# fix_ds_spacing = fix_spacing
# global_affine_ds = transform.apply_global_affine(
#     fix_ds, mov_ds,
#     fix_spacing, fix_spacing,
#     global_affine,)

In [ ]:
# # visualize specific ROIs and FISH images
# # 603(less spots in c0c3),124(edge),516(edge),660(fixed roi: only a small cut),827(not exsit)

# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()

# def ROI_edge(b1,b2,b3,b4,b5,b6):        
#     if b1 <0:
#         b1 = 0
#     if b3 <0:
#         b3 = 0  
#     if b5 <0:
#         b5 = 0
#     if b2 >= moving.shape[0]:
#         b2 = moving.shape[0] -1
#     if b4 >= moving.shape[1]:
#         b4 = moving.shape[1] -1 
#     if b6 >= moving.shape[2]:
#         b6 = moving.shape[2] -1     
#     return b1,b2,b3,b4,b5,b6

# froi_vis = 70
# mroi_vis = df.loc[df['fix'].astype(int) == froi_vis]['mov'].values[0].astype(int) #379

# AA=np.where(segmentation1==froi_vis)
# fixed_ROI = fixed[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
#           min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
#           min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
# print(fixed_ROI.shape)
# BB=np.where(segmentation2==mroi_vis)
# En_pixels = 15 # Enlarged pixels 
# b1 = min(BB[0])*zoom[0] - En_pixels
# b2 = max(BB[0])*zoom[0] + En_pixels
# b3 = min(BB[1])*zoom[1] - En_pixels
# b4 = max(BB[1])*zoom[1] + En_pixels
# b5 = min(BB[2])*zoom[2] - En_pixels
# b6 = max(BB[2])*zoom[2] + En_pixels
# #Edges evaluation

# b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
# moving_ROI = moving[b1:b2,b3:b4,b5:b6]
# print(moving_ROI.shape)
# viewer.add_image(fixed_ROI,colormap='green',blending='additive') #load image data into napari
# viewer.add_image(moving_ROI,colormap='red',blending='additive') #load image data into napari
# viewer.layers['fixed_ROI'].contrast_limits=(200, 300)
# viewer.layers['moving_ROI'].contrast_limits=(200, 300)


# fixed_ROI3 = fixed3[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
#           min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
#           min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]


# BB=np.where(segmentation2==mroi_vis)
# b1 = min(BB[0])*zoom[0] - En_pixels
# b2 = max(BB[0])*zoom[0] + En_pixels
# b3 = min(BB[1])*zoom[1] - En_pixels
# b4 = max(BB[1])*zoom[1] + En_pixels
# b5 = min(BB[2])*zoom[2] - En_pixels
# b6 = max(BB[2])*zoom[2] + En_pixels
# #Edges evaluation

# b1,b2,b3,b4,b5,b6 = ROI_edge(b1,b2,b3,b4,b5,b6)
# moving_ROI3 = moving3[b1:b2,b3:b4,b5:b6]
    
# viewer.add_image(fixed_ROI3,colormap='green',blending='additive') #load image data into napari
# viewer.add_image(moving_ROI3,colormap='red',blending='additive') #load image data into napari
# viewer.layers['fixed_ROI3'].contrast_limits=(200, 600)
# viewer.layers['moving_ROI3'].contrast_limits=(200, 600)

In [ ]:
# from libtiff import TIFF
# from scipy import misc
 
# ##tiff文件解析成图像序列
# ##tiff_image_name: tiff文件名；
# ##out_folder：保存图像序列的文件夹
# ##out_type：保存图像的类型，如.jpg、.png、.bmp等
# def tiff_to_image_array(tiff_image_name, out_folder, out_type): 
          
#     tif = TIFF.open(tiff_image_name, mode = "r")
#     idx = 0
#     for im in list(tif.iter_images()):
# 		#
#         im_name = out_folder + str(idx) + out_type
#         misc.imsave(im_name, im)
# #         print im_name, 'successfully saved!!!'
#         idx = idx + 1
#     return
 
# ##图像序列保存成tiff文件
# ##image_dir：图像序列所在文件夹
# ##file_name：要保存的tiff文件名
# ##image_type:图像序列的类型
# ##image_num:要保存的图像数目
# def image_array_to_tiff(image_dir, file_name, image_type, image_num):
 
#     out_tiff = TIFF.open(file_name, mode = 'w')
#     for i in range(0, image_num):
#         image_name = image_dir + str(i) + image_type
#         image_array = Image.open(image_name)
# #         img = image_array.resize((480, 480), Image.ANTIALIAS)
#         out_tiff.write_image(img, compression = None, write_rgb = True)
#     out_tiff.close()
#     return

In [ ]:
%%time
## load all ROIs
# roi_dir = seg_dir + 'GAD1_R1_R2.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
# roi_list = pd.read_csv(roi_dir,sep=',', index_col=0)
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()

# find all area of that fix ROI 
segmentation_mini=imread(seg_dir1+'mask_GAD11.tif')
roi = np.unique(segmentation_mini[segmentation_mini != 0])
roi = roi.astype(int)
roi = roi[np.where(roi!=[603])[0]]
roi = roi[np.where(roi!=[124])[0]]
roi = roi[np.where(roi!=[172])[0]] # few spots in c0 r2
roi = roi[np.where(roi!=[516])[0]]
roi = roi[np.where(roi!=[660])[0]]
roi = roi[np.where(roi!=[734])[0]] # only small part
roi = roi[np.where(roi!=[827])[0]]
print(roi)

roi_dir = seg_dir + 'allroi_matched.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
df = pd.read_csv(roi_dir)

spot_fix_c3_all = np.zeros((1, 4))
warp_spots_c3_all = np.zeros((1, 4))
spot_fix_c0_all = np.zeros((1, 4))
warp_spots_c0_all = np.zeros((1, 4))
fix_spacing=np.array([0.42,0.23,0.23])
i=0
Dist_sum_1 = np.zeros((len(roi), 4))

# all ROI
for aa in roi:
    
    # examplar ROI
#     aa = 505
    ROI_fixed = aa  #505
    print(f'fixed_ROI: #{ROI_fixed}')
    ROI_moving = df.loc[df['fix'].astype(int) == aa]['mov'].values[0].astype(int) #379
    print(f'moving_ROI: #{ROI_moving}')
    # run the ROI_ransac
    
    # #R2_C3 * R1_C3*; return new R1_C3
    threshold_fixed=0.0001 #0.001 for c3 fix mov 
    threshold_moving=0.0015 #0.0001 for c0 fix; 0.0005 for c0 moving;
    cc_r=12 
    match_threshold=0.7  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
    align_threshold=1
    distances_3,global_affine3, mov_affine3, fixed_ROI3, moving_ROI3, inv_affine3, inv_Transform3,fix_spots_new3,mov_spots_new3,fix_spots3, warp_spots_new3,ICP_affine3,ICP_affine_inv3,cc3 = ROI_affine(
        segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed3,moving3,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold)
    
    spot_fix = fix_spots3/fix_spacing + [2*cc3[0],4*cc3[1],4*cc3[2]]  # in phsical distance of S2
    spot_fix_c3 = np.column_stack((spot_fix,np.ones(len(spot_fix)).dot(aa)))
    spot_fix_c3_all = np.row_stack((spot_fix_c3_all,spot_fix_c3))
    warp_spots_3 = warp_spots_new3[:,:3]/fix_spacing + [2*cc3[0],4*cc3[1],4*cc3[2]]  # in pixels of S2
    warp_spots_new_c3 = np.column_stack((warp_spots_3,np.ones(len(warp_spots_new3)).dot(aa)))
    warp_spots_c3_all = np.row_stack((warp_spots_c3_all,warp_spots_new_c3))
    
    
    # #R2_C0 * R1_C0*; return new R1_C0
    threshold_fixed=0.00005 #0.001 for c3 fix mov 
    threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
    cc_r=12 
    match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
    align_threshold=3  # spots are sparsed
    
    ############################## apply c3 affine to c0 if track_affine == 1
    track_affine = 1
    
    distances_0,global_affine, mov_affine, fixed_ROI, mov_ds, moving_ROI, inv_affine,Transform,inv_Transform,fix_spots_new0,mov_spots_new0,fix_spots, warp_spots,warp_spots_new, ICP_affine,ICP_affine_inv,cc = ROI_affine_aligntrack(
            segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed,moving,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
            track_affine,global_affine3,inv_Transform3)
    
    # save every spots, global affine, warp affine.
    spot_fix = fix_spots/fix_spacing + [2*cc[0],4*cc[1],4*cc[2]]  # in phsical distance of S2
    spot_fix_c0 = np.column_stack((spot_fix,np.ones(len(spot_fix)).dot(aa)))
    spot_fix_c0_all = np.row_stack((spot_fix_c0_all,spot_fix_c0))
    warp_spots_0 = warp_spots_new[:,:3]/fix_spacing + [2*cc[0],4*cc[1],4*cc[2]]  # in pixels of S2
    warp_spots_new_c0 = np.column_stack((warp_spots_0,np.ones(len(warp_spots_new)).dot(aa)))
    warp_spots_c0_all = np.row_stack((warp_spots_c0_all,warp_spots_new_c0))
    # remember to remove the first 000 coordinates
    Dist_sum_1[i,:] = [ROI_fixed,distances_3,ROI_moving,distances_0]
        
#     roi_dir = 'ROI_affine/' + str(aa) + '/fix_c0.tif'
#     im_name = seg_dir + roi_dir
#     imsave(im_name, fixed_ROI)
    
#     roi_dir = 'ROI_affine/' + str(aa) + '/mov_c0.tif'
#     im_name = seg_dir + roi_dir
#     imsave(im_name, ICP_affine)

    a_rgb = np.zeros(fixed_ROI3.shape + (4,))
    a_rgb[..., 0] = fixed_ROI.astype(np.uint16)
    a_rgb[..., 1] = ICP_affine.astype(np.uint16)
    a_rgb[..., 2] = fixed_ROI3.astype(np.uint16)
    a_rgb[..., 3] = ICP_affine3.astype(np.uint16)
    roi_dir = 'ROI_affine/' + str(aa) + '_rgb.tif'
    im_name = seg_dir + roi_dir
    imsave(im_name, a_rgb)
    
    i = i + 1 
    print(f'Remaining ROI: {len(roi)- i}')
#     viewer.add_image(fixed_ROI,colormap='green',blending='additive') #load image data into napari
#     viewer.layers['fixed_ROI'].contrast_limits=(200, 600)
#     viewer.add_image(ICP_affine,colormap='magenta',blending='additive') #load image data into napari
#     viewer.layers['ICP_affine'].contrast_limits=(200, 600)
#     viewer.add_image(fixed_ROI3,colormap='green',blending='additive') #load image data into napari
#     viewer.layers['fixed_ROI3'].contrast_limits=(200, 3400)
#     viewer.add_image(ICP_affine3,colormap='magenta',blending='additive') #load image data into napari
#     viewer.layers['ICP_affine3'].contrast_limits=(200, 3400)

In [ ]:
%%time
## load all ROIs
# roi_dir = seg_dir + 'GAD1_R1_R2.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
# roi_list = pd.read_csv(roi_dir,sep=',', index_col=0)
# viewer = napari.view_image(data.astronaut(), rgb=True)
# napari.run()

# find all area of that fix ROI 
segmentation_mini=imread(seg_dir1+'mask_GAD11.tif')
roi = np.unique(segmentation_mini[segmentation_mini != 0])
roi = roi.astype(int)
roi = roi[np.where(roi!=[603])[0]]
roi = roi[np.where(roi!=[124])[0]]
roi = roi[np.where(roi!=[172])[0]] # few spots in c0 r2
roi = roi[np.where(roi!=[516])[0]]
roi = roi[np.where(roi!=[660])[0]]
roi = roi[np.where(roi!=[734])[0]] # only small part
roi = roi[np.where(roi!=[827])[0]]
print(roi)

roi_dir = seg_dir + 'allroi_matched.csv'   # directory to file containing the ROI metadata (neuron volume, etc.)
df = pd.read_csv(roi_dir)

spot_fix_c3_all = np.zeros((1, 4))
warp_spots_c3_all = np.zeros((1, 4))
spot_fix_c0_all = np.zeros((1, 4))
warp_spots_c0_all = np.zeros((1, 4))
fix_spacing=np.array([0.42,0.23,0.23])
i=0
Dist_sum_1 = np.zeros((len(roi), 4))

# all ROI
for aa in roi:
    
    # examplar ROI
#     aa = 85
    ROI_fixed = aa  #505
    print(f'fixed_ROI: #{ROI_fixed}')
    ROI_moving = df.loc[df['fix'].astype(int) == aa]['mov'].values[0].astype(int) #379
    print(f'moving_ROI: #{ROI_moving}')
    # run the ROI_ransac
    
    # #R2_C3 * R1_C3*; return new R1_C3
    threshold_fixed=0.0001 #0.001 for c3 fix mov 
    threshold_moving=0.0015 #0.0001 for c0 fix; 0.0005 for c0 moving;
    cc_r=12 
    match_threshold=0.7  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
    align_threshold=1
    distances_3,global_affine3, mov_affine3, fixed_ROI3, moving_ROI3, inv_affine3, inv_Transform3,fix_spots_new3,mov_spots_new3,fix_spots3, warp_spots_new3,ICP_affine3,ICP_affine_inv3,cc3 = ROI_affine(
        segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed3,moving3,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold)
    
    spot_fix = fix_spots3/fix_spacing + [2*cc3[0],4*cc3[1],4*cc3[2]]  # in phsical distance of S2
    spot_fix_c3 = np.column_stack((spot_fix,np.ones(len(spot_fix)).dot(aa)))
    spot_fix_c3_all = np.row_stack((spot_fix_c3_all,spot_fix_c3))
    warp_spots_3 = warp_spots_new3[:,:3]/fix_spacing + [2*cc3[0],4*cc3[1],4*cc3[2]]  # in pixels of S2
    warp_spots_new_c3 = np.column_stack((warp_spots_3,np.ones(len(warp_spots_new3)).dot(aa)))
    warp_spots_c3_all = np.row_stack((warp_spots_c3_all,warp_spots_new_c3))
    
    # #R2_C0 * R1_C0*; return new R1_C0
    threshold_fixed=0.00005 #0.001 for c3 fix mov 
    threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;
    cc_r=12 
    match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
    align_threshold=3  # spots are sparsed
    
    ############################## apply c3 affine to c0 if track_affine == 1
    track_affine = 1
    
    distances_0,global_affine, mov_affine, fixed_ROI, mov_ds, moving_ROI, inv_affine,Transform,inv_Transform,fix_spots_new0,mov_spots_new0,fix_spots, warp_spots,warp_spots_new, ICP_affine,ICP_affine_inv,cc = ROI_affine_aligntrack(
            segmentation1,segmentation2,ROI_fixed,ROI_moving,fixed,moving,threshold_fixed,threshold_moving,cc_r,match_threshold,align_threshold,
            track_affine,global_affine3,inv_Transform3)
    
    # save every spots, global affine, warp affine.
    spot_fix = fix_spots/fix_spacing + [2*cc[0],4*cc[1],4*cc[2]]  # in phsical distance of S2
    spot_fix_c0 = np.column_stack((spot_fix,np.ones(len(spot_fix)).dot(aa)))
    spot_fix_c0_all = np.row_stack((spot_fix_c0_all,spot_fix_c0))
    warp_spots_0 = warp_spots_new[:,:3]/fix_spacing + [2*cc[0],4*cc[1],4*cc[2]]  # in pixels of S2
    warp_spots_new_c0 = np.column_stack((warp_spots_0,np.ones(len(warp_spots_new)).dot(aa)))
    warp_spots_c0_all = np.row_stack((warp_spots_c0_all,warp_spots_new_c0))
    # remember to remove the first 000 coordinates
    
    Dist_sum_1[i,:] = [ROI_fixed,distances_3,ROI_moving,distances_0]
    i = i + 1 
    print(f'Remaining ROI: {len(roi)- i}')

In [ ]:
print(np.mean(Dist_sum_1[:-2,3]))
print(np.mean(Dist_sum_1[:-2,1]))

### napari viewer for single ROI

In [ ]:
%%time
# ROI_napari(mov_affine3, fixed_ROI3, moving_ROI3, fix_spots3, warp_spots_new3,ICP_affine3,fix_spacing)
# ROI_napari(mov_affine, fixed_ROI, moving_ROI, fix_spots, warp_spots_new,ICP_affine,fix_spacing)
viewer = napari.view_image(data.astronaut(), rgb=True)
napari.run()
viewer.add_image(fixed_ROI,colormap='green',blending='additive') #load image data into napari
viewer.add_image(moving_ROI,colormap='red',blending='additive') #load image data into napari
viewer.layers['fixed_ROI'].contrast_limits=(200, 600)
viewer.layers['moving_ROI'].contrast_limits=(200, 600)
viewer.add_image(mov_ds,colormap='yellow',blending='additive') #load image data into napari
viewer.layers['mov_ds'].contrast_limits=(200, 600)
viewer.add_image(mov_affine,colormap='yellow',blending='additive') #load image data into napari
viewer.layers['mov_affine'].contrast_limits=(200, 600)
viewer.add_image(ICP_affine,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['ICP_affine'].contrast_limits=(200, 600)
viewer.add_image(ICP_affine_inv,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['ICP_affine_inv'].contrast_limits=(200, 600)
s=fix_spots[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_fixed_match', size=1,
                  face_color='green',edge_color='green',blending='opaque') 
C = warp_spots_new
viewer.add_points(np.transpose(np.array([C[:,0]/fix_spacing[0],C[:,1]/fix_spacing[1],C[:,2]/fix_spacing[2]])),name ='warp_spots_new', size=1,
                  face_color='yellow',edge_color='yellow',blending='opaque') 
C = warp_spots
viewer.add_points(np.transpose(np.array([C[:,0]/fix_spacing[0],C[:,1]/fix_spacing[1],C[:,2]/fix_spacing[2]])),name ='C', size=1,
                  face_color='yellow',edge_color='yellow',blending='opaque') 

viewer.add_image(fixed_ROI3,colormap='green',blending='additive') #load image data into napari
viewer.add_image(moving_ROI3,colormap='red',blending='additive') #load image data into napari
viewer.layers['fixed_ROI3'].contrast_limits=(200, 3400)
viewer.layers['moving_ROI3'].contrast_limits=(200, 3400)
# viewer.add_image(mov_affine3,colormap='yellow',blending='additive') #load image data into napari
# viewer.layers['mov_affine3'].contrast_limits=(200, 3400)
viewer.add_image(ICP_affine3,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['ICP_affine3'].contrast_limits=(200, 3400)
# viewer.add_image(ICP_affine_inv3,colormap='magenta',blending='additive') #load image data into napari
# viewer.layers['ICP_affine_inv3'].contrast_limits=(200, 3400)
# s=fix_spots3[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
# viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c3_fixed_match', size=1,
#                   face_color='green',edge_color='green',blending='opaque') 
# C3 = warp_spots_new3
# viewer.add_points(np.transpose(np.array([C3[:,0]/fix_spacing[0],C3[:,1]/fix_spacing[1],C3[:,2]/fix_spacing[2]])),name ='C3', size=1,
#                   face_color='yellow',edge_color='yellow',blending='opaque') 

In [ ]:
A = fix_spots[:,:3]
print(A[0]) 
B = warp_spots[:,:3]
print(B[0]) 

Transform, distances1 = icp(A, B)
inv_Transform, distances2 = icp(B, A)
    
p = np.append(B, np.ones((B.shape[0],1)), axis=1)
print(p.dot(inv_Transform.T)[:,:3][0])
print(p.dot(Transform.T)[:,:3][0]) 

print(np.mean(cloud_distance(p.dot(inv_Transform.T)[:,:3],B)))
print(np.mean(cloud_distance(p.dot(Transform.T)[:,:3],B)))


p = np.append(A, np.ones((A.shape[0],1)), axis=1)
print(p.dot(inv_Transform.T)[:,:3][0])
print(p.dot(Transform.T)[:,:3][0]) 

print(np.mean(cloud_distance(p.dot(inv_Transform.T)[:,:3],A)))
print(np.mean(cloud_distance(p.dot(Transform.T)[:,:3],A)))

In [ ]:
mov_affine_0 = transform.apply_global_affine(
fixed_ROI, moving_ROI,
fix_spacing, fix_spacing,
global_affine3,)

mov_ds = transform.apply_global_affine(
fixed_ROI, mov_affine_0,
fix_spacing, fix_spacing,
inv_Transform3,) 

viewer.add_image(mov_affine_0,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['mov_affine_0'].contrast_limits=(200, 400)

viewer.add_image(mov_ds,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['mov_ds'].contrast_limits=(200, 400)

### Napari view all spots inside ROIs

In [ ]:
viewer = napari.view_image(data.astronaut(), rgb=True)
napari.run()
# xyz 1882 2500 2500 
zoom=[2,4,4]
s=spot_fix_c0_all[:,:3]#convert spot physical coordinates to pixel coordinates
#load spots into napari, pixel positions in xyz order
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_fixed', size=1,
                  face_color='yellow',edge_color='yellow',blending='opaque') 
s=warp_spots_c0_all[:,:3]#convert spot physical coordinates to pixel coordinates
#load spots into napari, pixel positions in xyz order
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_warp', size=1,
                  face_color='green',edge_color='green',blending='opaque') 

s=spot_fix_c3_all[:,:3]#convert spot physical coordinates to pixel coordinates
#load spots into napari, pixel positions in xyz order
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c3_fixed', size=1,
                  face_color='yellow',edge_color='yellow',blending='opaque') 
s=warp_spots_c3_all[:,:3]#convert spot physical coordinates to pixel coordinates
#load spots into napari, pixel positions in xyz order
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c3_warp', size=1,
                  face_color='green',edge_color='green',blending='opaque') 

# plot images
planes = 0
a=50
fix_ds = fixed3[planes:planes+a ,:,:]
mov_ds = moving3[planes:planes+a ,:,:]
fix_spacing=np.array([0.42,0.23,0.23])
zoom_image=np.array([1,1,1])
fix_ds_spacing = fix_spacing/zoom_image
viewer.add_image(fix_ds,colormap='green',blending='additive') 
viewer.layers['fix_ds'].contrast_limits=(200, 1000)
viewer.add_image(mov_ds,colormap='red',blending='additive') 
viewer.layers['mov_ds'].contrast_limits=(200, 1000)

In [ ]:
# distances, indices = nearest_neighbor(warp_spots_new[:,0:3],fix_spots[:,0:3])
# print(distances)
# print(indices) 
# print(warp_spots_new[:,0:3].shape)

# CPD

In [ ]:
import copy
import numpy as np
import open3d as o3
from probreg import cpd
import transforms3d as t3d

def estimate_normals(pcd, params):
    pcd.estimate_normals(search_param=params)
    pcd.orient_normals_to_align_with_direction()# load source and target point cloud

source = o3.io.read_point_cloud('D:/0_software/github/probreg/examples/bunny.pcd')
source.remove_non_finite_points()
target = copy.deepcopy(source)

# read points of # point cloud
# xyz = np.asarray(result.points)

A = fix_spots_new0
B = mov_spots_new0

# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
# pcd = o3.geometry.PointCloud()
target.points = o3.utility.Vector3dVector(A)
source.points = o3.utility.Vector3dVector(B)
# o3.io.write_point_cloud("../../test_data/sync.ply", pcd)
# o3.visualization.draw_geometries(pcd)

# register
tf_param, _, _ = cpd.registration_cpd(source, target)
result = copy.deepcopy(source)
result.points = tf_param.transform(result.points)

# draw result
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0.5, 1])
# o3.visualization.draw_geometries([result,source])

s=np.asarray(source.points)[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_src', size=1,
                  face_color='green',edge_color='green',blending='opaque')
s=np.asarray(target.points)[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_tar', size=1,
                  face_color='yelow',edge_color='yellow',blending='opaque')
s=np.asarray(result.points)[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c0_cpd', size=1,
                  face_color='magenta',edge_color='magenta',blending='opaque')

# ICP

In [ ]:
source.paint_uniform_color([1, 0, 0])
target.paint_uniform_color([0, 1, 0])
result.paint_uniform_color([0, 0.5, 1])
threshold = 0.05 # how to set initially
icp_iteration = 100
for i in range(icp_iteration):
    reg_p2p = o3.pipelines.registration.registration_icp(result, target, threshold,
                np.identity(4), o3.pipelines.registration.TransformationEstimationPointToPoint(),
                o3.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
    result.transform(reg_p2p.transformation)
#     vis.update_geometry(source)
#     vis.update_geometry(target)
#     vis.update_geometry(result)
#     vis.poll_events()
# vis.run()
o3.visualization.draw_geometries([result,source,target])

In [ ]:
#Bigstitcher
# R_dir ='E:/Maxprobe_analysis/R2_R1_3tm50/Bigstitcher'
# imsave(R_dir + '/ID#5/fixed_ROI3.tif', fixed_ROI3)
# imsave(R_dir + '/ID#5/moving_ROI3.tif', moving_ROI3)
# imsave(R_dir + '/ID#5/fixed_ROI.tif', fixed_ROI)
# imsave(R_dir + '/ID#5/moving_ROI.tif', moving_ROI)

# global_affine_bs=np.eye(4)
# global_affine_bs[0]=[1.0003, 0.0194, -0.059, 1.7654]
# global_affine_bs[1]=[-0.031, 1.0568, -0.0156, -10.1989]
# global_affine_bs[2]=[-0.0035, -0.0051, 0.9936, 0.5553]
# # Scaling: 1.0008, 1.057, 0.9955
# bs_affine = transform.apply_global_affine(
# moving_ROI3,fixed_ROI3,
# fix_spacing, fix_spacing,
# global_affine_bs,)
# viewer.add_image(bs_affine,colormap='magenta',blending='additive') #load image data into napari
# viewer.layers['bs_affine'].contrast_limits=(200, 3400)

# # fix_spots_new3
# B = fix_spots_new3
# p = np.append(B, np.ones((B.shape[0],1)), axis=1)
# C = p.dot(global_affine_bs.T)
# s=C[:,:3]/fix_spacing#convert spot physical coordinates to pixel coordinates
# viewer.add_points(np.transpose(np.array([s[:,0],s[:,1],s[:,2]])),name ='c3_fixed_BS_match', size=1,
#                   face_color='magenta',edge_color='magenta',blending='opaque') 

distances = distances_3
fig=plt.figure(dpi=120,figsize=(2,3))
plt.violinplot(distances)
sns.despine() 
plt.xticks([])
plt.xlabel('Spots:'+ str(distances.shape[0]))
plt.ylabel('Distance/um')
ave=np.average(distances)
plt.title(str(float('%.2f' % ave)))
# plt.axis('off')
plt.show()
plt.tight_layout()
viewer.add_image(ICP_affine3,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['ICP_affine3'].contrast_limits=(200, 3400)

distances = distances_0
fig=plt.figure(dpi=120,figsize=(2,3))
plt.violinplot(distances)
sns.despine() 
plt.xticks([])
plt.xlabel('Spots:'+ str(distances.shape[0]))
plt.ylabel('Distance/um')
ave=np.average(distances)
plt.title(str(float('%.2f' % ave)))
# plt.axis('off')
plt.show()
plt.tight_layout()
viewer.add_image(ICP_affine,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['ICP_affine'].contrast_limits=(200, 3400)

# Test efficiency

### Colocalization across rounds: R2/R1 C3; C0;

In [ ]:
neighbor_radius1 = 6
print('colocalization of R2/R1 C3 spots')
lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix_c3_all[:,:3],warp_spots_c3_all[:,:3],neighbor_radius1)  # return in pixel
print(np.mean(eucldist(lipo_c0,lipo_c1)))

# print('colocalization of R2/R1 C0 spots')
# lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix_c0_all[:,:3],warp_spots_c0_all[:,:3],neighbor_radius1)  # return in pixel
# print(np.mean(eucldist(lipo_c0,lipo_c1)))

fig=plt.figure(dpi=120,figsize=(2,3))
plt.violinplot(eucldist(lipo_c0,lipo_c1))
sns.despine() 
plt.xticks([])
plt.ylim([0,neighbor_radius1*2])
plt.xlabel('Spots:'+ str(eucldist(lipo_c0,lipo_c1).shape[0]))
plt.ylabel('Distance/pixel')
ave=np.average(eucldist(lipo_c0,lipo_c1))
plt.title(str(float('%.2f' % ave)))
# plt.axis('off')
plt.show()
plt.tight_layout()

### Colocalization SAME rounds: R2 C3/C0;Non-registered; Registered R1 C3/C0;

In [ ]:
print('colocalization of R2 C3/C0 spots')
lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
print(eucldist(lipo_c0,lipo_c1))

print('colocalization of Non-registered R1 spots')
lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
print(eucldist(lipo_c0,lipo_c1))

print('colocalization of registered R1 spots') 
lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel
print(eucldist(lipo_c0,lipo_c1))

fig=plt.figure(dpi=120,figsize=(2,3))
plt.violinplot(eucldist_vectorized(lipo_c0,lipo_c1))
sns.despine() 
plt.xticks([])
plt.ylim([0,neighbor_radius1*2])
plt.xlabel('Spots:'+ str(eucldist_vectorized(lipo_c0,lipo_c1).shape[0]))
plt.ylabel('Distance/pixel')
ave=np.average(eucldist_vectorized(lipo_c0,lipo_c1))
plt.title(str(float('%.2f' % ave)))
# plt.axis('off')
plt.show()
plt.tight_layout()

### For handling multiple channels

In [ ]:
# planes = 0
# a=50
# fix_ds = fixed3[planes:planes+a ,:,:]
# # mov_ds = moving3[planes:planes+a ,:,:]
# fix_spacing=np.array([0.42,0.23,0.23])
# # zoom=np.array([0.5,0.25,0.25])
# zoom_image=np.array([1,1,1])
# fix_ds_spacing = fix_spacing/zoom_image

# # plot images
# viewer.add_image(fix_ds,colormap='green',blending='additive') 
# viewer.layers['fix_ds'].contrast_limits=(200, 1000)
# # viewer.add_image(mov_ds,colormap='red',blending='additive') 
# # viewer.layers['mov_ds'].contrast_limits=(200, 1000)

# # global_affine_all = affine.ransac_affine(
# #     fix_ds, mov_ds,
# #     fix_ds_spacing, fix_ds_spacing,
# #     min_radius=6, max_radius=12, match_threshold=0.40,
# #     cc_radius=12, 
# # )

# # # or from the bigstream results
# # print(global_affine_all)
# global_affine_all=np.eye(4)
# global_affine_all[0]=[ 1.01452988, -0.01885208,  -0.03336844,  18.66071098]
# global_affine_all[1]=[0.09519188,   1.00508661,  -0.09667662, -17.6666447 ]
# global_affine_all[2]=[-0.07341534,  0.06896605,   0.97213685,   9.59959212]

# # apply the global affine to the moving image
# mov_affine_all = transform.apply_global_affine(
#     fix_ds, mov_ds,
#     fix_ds_spacing, fix_ds_spacing,
#     global_affine_all,
# )

# viewer.add_image(mov_affine_all,colormap='magenta',blending='additive') 
# viewer.layers['mov_affine_all'].contrast_limits=(200, 1000)

## ROI_Ransac from Bigstream
### Spots can be choosed from bigstream extracted spots

In [ ]:
# Airlocolize spots data
fix_spacing=np.array([0.42,0.23,0.23])
mov_spacing=fix_spacing
# fix_ds_spacing = fix_spacing/zoom
min_radius=6
max_radius=12
cc_r=12
cc_radius=cc_r # used for radius pixel of context information. 
match_threshold=0.5  #0.7 for c3, 0.4 for c0c3, 0.5 for c0
nspots=30000
num_sigma_max=6
align_threshold=3.0
threshold_fixed=0.0001 #0.001 for c3 fix mov 
threshold_moving=0.0005 #0.0001 for c0 fix; 0.0005 for c0 moving;

AA=np.where(segmentation1==ROI_fixed)
fixed_ROI = fixed[min(AA[0])*zoom[0]:max(AA[0])*zoom[0],
          min(AA[1])*zoom[1]:max(AA[1])*zoom[1],
          min(AA[2])*zoom[2]:max(AA[2])*zoom[2]]
print(fixed_ROI.shape)

BB=np.where(segmentation1==ROI_moving)
moving_ROI = moving[min(BB[0])*zoom[0]:max(BB[0])*zoom[0],
          min(BB[1])*zoom[1]:max(BB[1])*zoom[1],
          min(BB[2])*zoom[2]:max(BB[2])*zoom[2]]
print(moving_ROI.shape)
fix_ds = fixed_ROI
mov_ds = moving_ROI
# get spots
print('Getting key points')

# use import spots or not. If yes, the spot should align with images checked with Napari
hAir = 0
# Adapt ROI from images or from hAirlocalize
if hAir < 1:
    print(f'hAirlocalize points')
    # read spot data into memory as numpy arrays  Airlocolize spots data in xyz order: physical distance
    spotdir = 'E:/Maxprobe_analysis/R2_R1_3tm50/R2_3tm50_1920/spots/R2_c3_ROI.txt'
    spot_fix=np.loadtxt(spotdir, delimiter=',')
    fixed_spots1 = spot_fix[spot_fix[:,4] == i][:,:3]
    spotdir = 'E:/Maxprobe_analysis/R2_R1_3tm50/R1_3tm50_1920/spots/R1_c3_ROI.txt'
    spot_mov=np.loadtxt(spotdir, delimiter=',')
    moving_spots1 = spot_mov[spot_mov[:,4] == i][:,:3]
    # change to zyx order
    fixed_spots11 = np.transpose(np.array([fixed_spots1[:,2],fixed_spots1[:,1],fixed_spots1[:,0]]))
    moving_spots11 = np.transpose(np.array([moving_spots1[:,2],moving_spots1[:,1],moving_spots1[:,0]]))
    # convert to physical units
    ccc = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
    fix_spots = (fixed_spots11 - ccc * fix_spacing)
    fix_spots_new = fix_spots
    ns = fix_spots.shape[0]
    print(f'FIXED image: found {ns} key points')

    dd = [min(AA[0])*zoom[0],min(AA[1])*zoom[1],min(AA[2])*zoom[2]]
#     mov_spots_h = (moving_spots11/fix_spacing - dd).astype(int)
    mov_spots = (moving_spots11 - dd * mov_spacing)
    ns = mov_spots.shape[0]
    print(f'MOVING image: found {ns} key points')
    
    # get contexts
    fix_spots0 = features.get_spot_context(
        fixed3, fixed_spots11, fix_spacing, cc_r,
    )
    mov_spots0 = features.get_spot_context(
        moving3, moving_spots11, mov_spacing, cc_r,
    )

    print(f'get contexts')

    # get point correspondences may change to mutual information
    correlations = features.pairwise_correlation(
        fix_spots0, mov_spots0,
    )
    print(f'correlations')
    fix_spots1, mov_spots1 = features.match_points(
        fix_spots0, mov_spots0,
        correlations, match_threshold,
    )

    ns = fix_spots1.shape[0]
    print(f'MATCHED points: found {ns} matched points')

    global_affine = ransac.ransac_align_points(fix_spots1 - cc * fix_spacing, mov_spots1 - dd * mov_spacing, align_threshold,)
    print(global_affine)

    inv_affine = ransac.ransac_align_points(mov_spots1 - dd * mov_spacing, fix_spots1 - cc * fix_spacing, align_threshold,)
#     print(inv_affine)

# functions for applying transforms are in bigstream.transform. apply the global affine to the moving image
mov_affine = transform.apply_global_affine(
    fix_ds, mov_ds,
    fix_spacing, fix_spacing,
    global_affine,
)

### We use ICP to register point clouds of an example ROI dataset. 
#### spots data were generated by Airlocolize of the multiFISH pipeline
These spots have not been registered at all?

In [ ]:
#transform spots
if __name__ == "__main__":
    A = fix_spots
    B = mov_spots
    B = warp_spots_new[:,:3]
    #     A = np.random.randint(0,101,(20,3))  # 20 points for test
    #     rotz = lambda theta: np.array([[np.cos(theta),-np.sin(theta),0],
    #                                        [np.sin(theta),np.cos(theta),0],
    #                                        [0,0,1]])
    #     trans = np.array([2.12,-0.2,1.3])
    #     B = A.dot(rotz(np.pi/4).T) + trans 

    Transform, distances = icp(A, B)
    inv_Transform, distances = icp(B, A)
    print(np.mean(distances))
    np.set_printoptions(precision=3,suppress=True)
    print (Transform)
    # functions for applying transforms are in bigstream.transform. apply the global affine to the moving image
    ICP_affine = transform.apply_global_affine(
        fix_ds, mov_affine,
        fix_spacing, fix_spacing,
        Transform,)
    #     print (inv_Transform)
    p = np.append(B, np.ones((B.shape[0],1)), axis=1)
    C = p.dot(inv_Transform.T)

    #     viewer = napari.view_image(data.astronaut(), rgb=True)
    #     napari.run()
    viewer.add_image(ICP_affine,colormap='red',blending='additive') #load image data into napari
    viewer.layers['ICP_affine'].contrast_limits=(200, 3400)  

    viewer.add_points(np.transpose(np.array([A[:,0]/fix_spacing[0],A[:,1]/fix_spacing[1],A[:,2]/fix_spacing[2]])),name ='A', size=1,
                      face_color='green',edge_color='green',blending='opaque') 
    viewer.add_points(np.transpose(np.array([B[:,0]/fix_spacing[0],B[:,1]/fix_spacing[1],B[:,2]/fix_spacing[2]])),name ='B', size=1, 
                      face_color='red',edge_color='red',blending='opaque') 
    viewer.add_points(np.transpose(np.array([C[:,0]/fix_spacing[0],C[:,1]/fix_spacing[1],C[:,2]/fix_spacing[2]])),name ='C', size=1,
                      face_color='yellow',edge_color='yellow',blending='opaque') 

In [ ]:
# Test ICP distance change. Minor increased performance
spot_fix = fix_spots
warp_spots_new = get_spot_inside(warp_spots,segmentation_mini,fix_spacing,ROI_fixed,cc)
spot_mov = C
lipo_c0,lipo_c1,true_pos_c0,true_pos_c1,dist = colocalization(spot_fix,spot_mov,neighbor_radius1)  # return in pixel

# find and remove spots outside of this ROI
c0=spot_fix[:,:3].copy()
c1=spot_mov[:,:3].copy()
kdtree_c0 = cKDTree(c0)
kdtree_c1 = cKDTree(c1)
dist2,idx2 = kdtree_c0.query(c1, k=3)

fig=plt.figure(dpi=120,figsize=(2,3))
plt.violinplot(dist2[:,0])
sns.despine() 
plt.xticks([])
plt.ylim([0,neighbor_radius1*2])
plt.xlabel('Spots:'+ str(dist2[:,0].shape[0]))
plt.ylabel('Distance/pixel')
ave=np.average(dist2[:,0])
plt.title(str(float('%.2f' % ave)))
# plt.axis('off')
plt.show()
plt.tight_layout()

### Need correlational method that calculates the mutual information, not only the intensity of blobs. 

In [ ]:
# grab and flatten context
a_con = np.array( [a[1].flatten() for a in fix_spots0] , dtype=object)
b_con = np.array( [b[1].flatten() for b in mov_spots0] , dtype=object)

# get means and std for all contexts, center contexts
a_mean, a_std = _stats(a_con)
b_mean, b_std = _stats(b_con)
a_con = a_con - a_mean[..., None]
b_con = b_con - b_mean[..., None]

# compute pairwise correlations
corr = np.matmul(a_con, b_con.T)
corr = corr / a_std[..., None]
corr = corr / b_std[None, ...]
corr = corr / a_con.shape[1]

# contexts with no variability are nan, set to 0
corr[np.isnan(corr)] = 0
correlations = corr

In [ ]:
viewer.add_image(R1C3_aligned,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['R1C3_aligned'].contrast_limits=(200, 1400)

In [ ]:
imsave(seg_dir + 'R1C3_aligned_505_0.7_8.tif',R1C3_aligned)
# imsave(seg_dir + 'R2C3_ROI.tif',R2C3_ROI)
# imsave(seg_dir + 'R1C3_ROI.tif',R1C3_ROI)

In [ ]:
viewer.add_image(R2C3_ROI,colormap='green',blending='additive') #load image data into napari
viewer.add_image(R1C3_ROI,colormap='red',blending='additive') #load image data into napari
viewer.add_image(R1C3_aligned,colormap='magenta',blending='additive') #load image data into napari
viewer.layers['R2C3_ROI'].contrast_limits=(200, 1400)
viewer.layers['R1C3_ROI'].contrast_limits=(200, 1400)
viewer.layers['R1C3_aligned'].contrast_limits=(200, 1400)

### Visualize aligned images

In [ ]:
viewer.add_image(R2C3_ROI,colormap='green',blending='additive') #load image data into napari
viewer.add_image(R1C3_aligned,colormap='red',blending='additive') #load image data into napari
viewer.add_image(R2C0_aligned,colormap='magenta',blending='additive') #load image data into napari
viewer.add_image(R1C0_aligned,colormap='yellow',blending='additive') #load image data into napari
viewer.layers['R2C3_ROI'].contrast_limits=(200, 1400)
viewer.layers['R1C3_aligned'].contrast_limits=(200, 1400)
viewer.layers['R2C0_aligned'].contrast_limits=(200, 400)
viewer.layers['R1C0_aligned'].contrast_limits=(200, 400)

## Another round of registration

In [ ]:
# with the same parameters will not make the images getting better registered. Removed it.
# apply the global affine to the moving image
global_affine2 = affine.ransac_affine(
    R2C3_ROI, R1C3_aligned,
    fix_ds_spacing, fix_ds_spacing,
    min_radius=2, max_radius=8, match_threshold=0.5,
    cc_radius=8
)
# functions for applying transforms are in bigstream.transform
# apply the global affine to the moving image
R1C3_aligned2 = transform.apply_global_affine(
    R2C3_ROI, R1C3_aligned,
    fix_ds_spacing, fix_ds_spacing,
    global_affine2,
)

viewer.add_image(R1C3_aligned2,colormap='red',blending='additive') #load image data into napari
viewer.layers['R1C3_aligned2'].contrast_limits=(200, 1400)    

## Deformable registration
# Not find a good way to get fine results yet

In [ ]:
from bigstream import deform
warps = deform.deformable_align(
    fix_ds, mov_affine,
    fix_ds_spacing, fix_ds_spacing,
    iterations=[500,250,100,1],
    shrink_factors=[8,4,2,1],
    smooth_sigmas=[16,8,4,2],
)
    
from CircuitSeeker import defreg as csdr
# local_warps = local_warps.squeeze()

aligned = csdr.applyTransformToImage(
    fix_ds, mov_affine,
    fix_ds_spacing, fix_ds_spacing,
    transform_list=[warps,]
)
print(aligned.shape)
viewer.add_image(aligned,colormap='red',blending='additive') #load image data into napari
viewer.layers['aligned'].contrast_limits=(200, 1400)   